In [ ]:
import sys, os
import time
import cv2

import pickle, bz2 

import tensorflow as tf
import numpy as np

sys.path.append(os.path.join(os.path.abspath(''), '../../utils'))

from diambraImitationLearning import *

In [ ]:
hwc_dim = [256,256,7]
n_actions = [9, 8]
RLTrajFileList = ["./RLTraj_2020-08-31_23-35-01"]
rank = 0
totalCpus = 1

In [ ]:
env = diambraImitationLearning(hwc_dim, n_actions, RLTrajFileList, rank, totalCpus)

In [ ]:
observation = env.reset()
env.trajSummary()

In [ ]:
shp = observation.shape

additionalPar = int(observation[0,0,shp[2]-1])
# 1P
nScalarAddPar = additionalPar - 2*env.numberOfCharacters - env.actBufLen*(env.n_actions[0]+env.n_actions[1])

print("Additional Par = ", additionalPar)
print("N scalar actions = ", nScalarAddPar)

cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 10
currNumEp = 0

while currNumEp < maxNumEp:
    
    observation, reward, done, action = env.step(0)
    print("Action:", action)
    
    addPar = observation[:,:,shp[2]-1]
    addPar = np.reshape(addPar, (-1))
    addPar = addPar[1:additionalPar+1]
    actions = addPar[0:additionalPar-nScalarAddPar-2*env.numberOfCharacters]
        
    limAct = [env.actBufLen * env.n_actions[0], 
              env.actBufLen * env.n_actions[0] + env.actBufLen * env.n_actions[1]]
        
    moveActionsP1   = actions[0:limAct[0]]
    attackActionsP1 = actions[limAct[0]:limAct[1]]
    moveActionsP1   = np.reshape(moveActionsP1, (env.actBufLen,-1))
    attackActionsP1 = np.reshape(attackActionsP1, (env.actBufLen,-1))
    print("Move actions P1 = ", moveActionsP1)
    print("Attack actions P1 = ", attackActionsP1)    
        
    others = addPar[additionalPar-nScalarAddPar-2*env.numberOfCharacters:]
    print("ownHealth = ", others[0])
    print("oppHealth = ", others[1])
    print("ownPosition = ", others[2])
    print("oppPosition = ", others[3])
    #print("stage = ", others[4])
        
    obs = np.array(observation).astype(np.float32)
    
    for idx in range(shp[2]-1):
        cv2.imshow("image"+str(idx), obs[:,:,idx])
    
    cv2.waitKey()
 
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0

        observation = env.reset()
        input("Stop")

print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       